Для обработки опубликованных и изданных текстов А. П. Чехова. При обработке неизданных/неопубликованных документов в исходном файле (TEIdoc) изменить информацию внутри тегов "isFinished" (оконченные/неокончнные) и "isPublished" (опубликованные/неопубликованные).

Исходные HTML-файлы должны быть размещены на гугл-диске (если они расположены в другом месте, поменять директорию. Переменные directory, new_directory).

При обработке томов, содержащих только один жанр текста (например, том 11 - пьесы) заполнить в исходном файле (TEIdoc) тег "textClass" (тип текста).

In [ ]:
!pip install natasha
# Для документов, расположенных на гугл-диске
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from bs4 import BeautifulSoup
import re
from pathlib import Path

from natasha import (
    Segmenter, MorphVocab,
    NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger,
    NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor,
    PER, Doc
)

In [ ]:
BASE_PATH = Path('drive/My Drive/Colab Notebooks/Transform_tei/HTML_TEI')
Texts_HTML_PATH = BASE_PATH / 'texts_html'
Texts_TEI_PATH = BASE_PATH / 'texts_tei'
TEI_Preform_file = BASE_PATH / 'TEIdoc.xml'
NOTES_Preform_PATH = BASE_PATH / 'notes_html'

In [ ]:
your_name = 'Кудин Анастасией' #имя заполняющего

In [ ]:
def open_with_soup(file_path, encoding):
    with open(file_path,'r',encoding=encoding) as file:
        content = file.read()
    soup = BeautifulSoup(content, 'html.parser')
    return soup


def create_dir(directory):
  if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)


def extract_names(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        if span.type == PER:
            span.normalize(morph_vocab)
            span.extract_fact(names_extractor)
    names = [{'normal': _.normal, 'fio': _.fact.as_dict, 'start': _.start, 'end': _.stop} for _ in doc.spans if _.fact]
    return names


def extract_dates(text):
    return list(dates_extractor(text))

In [ ]:
class MetaSetter():
    """Класс с методами для заполнения метаинформации о произведении"""
    def __init__(self, soup_html, soup_tei):
        self.soup_html = soup_html
        self.soup_tei = soup_tei


    def fill_title(self):
        """Заполняет название"""
        title_main_tei = self.soup_tei.find('title')
        title_tag_html = self.soup_html.find_all('meta', attrs={'name':'title'})
        title_main_tei.string = title_tag_html[0].get('content')
        if len(title_tag_html) == 2: # Если есть подзаголовок
            title_main_tei.string += f" ({title_tag_html[1].get('content')})"


    def fill_description(self):
        """Заполняет описание текста"""
        description_html = self.soup_html.find('div', class_='description')
        full_bibl = self.soup_tei.find('biblfull').find('p')
        full_bibl.string = description_html.text
        self._description_html = description_html


    def fill_transformator_name(self, transformator_name):
        """Заполняет имя того, кто преобразовал текст в TEI"""
        name_resp_stmt = self.soup_tei.find('respstmt').find('persname')
        name_resp_stmt.string = transformator_name


    def fill_size_info(self):
        """Заполняет информацию об объеме произведения"""
        extent_inf = self.soup_tei.find('extent')
        # Получаем список номеров страниц
        list_ids = [] 
        for tag in self.soup_html.find_all('span', class_='page') :
            tag_page=tag.get('id')
            list_ids.append(tag_page[2:])

        # Считаем объем
        volume = int(list_ids[-1]) - int(list_ids[0]) + 1
        volume_str = str(volume)

        # Формируем результирующую строку
        if volume_str[-1] in ['5', '6', '7', '8', '9', '0'] or\
                (len(volume_str) == 2 and volume_str[-2] == '1'):
            volume_f = volume_str + ' страниц'
        elif volume_str[-1] == '1':
            volume_f = volume_str + ' страница'
        else:
            volume_f = volume_str + ' страницы'

        # Заполняем информацию об объеме в атрибутах тегов
        tag_measure = self.soup_tei.new_tag('measure') 
        extent_inf.insert(0, tag_measure)

        self.soup_tei.find('measure')['unit'] = 'pages'
        self.soup_tei.find('measure')['quantity'] = volume_str

        measure_inf = self.soup_tei.find('measure')
        measure_inf.string = volume_f


    def fill_publication_date(self):
        """Заполняет дату публикации произведения"""
        publ_date = self.soup_tei.find('publicationstmt')
        publ_date1 = publ_date.find('date')
        date_1 = self.soup_html.find('meta', attrs={'name':'date'})
        publ_date1.string = date_1.get('content')
        publ_date.find('date')['when'] = date_1.get('content')


    def fill_creation_date(self):
        """Заполняет дату создания произведения"""
        date_cr = self.soup_tei.find('creation').find('date')
        abz = self._description_html.find_all('p')
        search = abz[-2].text

        date_from_to = re.findall(r'\d{4}—\d{4}', search)
        if len(date_from_to) > 0: # Если том создавался несколько лет
            date_cr.string = date_from_to[0]
            date_cr['from'] = date_from_to[0][:4]
            date_cr['to'] = date_from_to[0][-4:]
        else: # Если том создавался один год (5 и 6 тома)
            date_when = re.findall(r'\d{4}', search)[0]
            date_cr['when'] = date_when
            date_cr.string = date_when
        self._abz = abz


    def fill_edition_info(self):
        """Заполняет информацию об издании"""
        edstmt = self._abz[0].text.split('//')
        edstmt1 = edstmt[1].split('\n')[0]
        edition_stmt = self.soup_tei.find('editionstmt').find('p')
        edition_stmt.string = edstmt1
        self._edition_stmt = edition_stmt


    def fill_volume_num(self):
        """Заполняет номер тома"""
        volume = re.findall(r'Т. \d+.', self._edition_stmt.string)
        volume_n = re.findall(r'\d+', volume[0])
        bibl_sc = self.soup_tei.find('biblscope')
        bibl_sc.string = 'Том ' + volume_n[0]


    def fill_all_meta(self, transformator_name):
        """Полностью заполняет метаинформацию"""
        #Порядок выполнения функций лучше не менять
        self.fill_title()
        self.fill_description()
        self.fill_transformator_name(transformator_name)
        self.fill_size_info()
        self.fill_publication_date()
        self.fill_creation_date()
        self.fill_edition_info()
        self.fill_volume_num()

In [ ]:
class ExistenceSeter():
    """Класс с методами для разметки сущностей"""
    
    @staticmethod
    def set_people(par_tag, location_text):
        """Расставляет теги имен"""

        def pattern_repl_name_fio(name_part, name_info_fio, name_format):
            """Готовит шаблоны для замены"""
            if name_part in name_info_fio:
                pattern_start = name_info_fio[name_part]
                pattern_end = r'.'
                repl_end = r'.'
                if len(name_info_fio[name_part]) != 1:
                    try:
                        pattern_start = name_info_fio[name_part][:-2]
                    except:
                        pattern_start = name_info_fio[name_part][:-1]
                    pattern_end = r'(\w*)'
                    repl_end = r'\g<1>'
                return pattern_start + pattern_end, name_format.format(name_info_fio[name_part], pattern_start + repl_end)
            return None

        names = extract_names(str(par_tag.string))
        person_format = '<PersName type="{0}">{1}</PersName>'
        name_format = '<forename xml:id="{0}">{1}</forename>'
        surname_format = '<surname xml:id="{0}">{1}</surname>'
        middlename_format = '<forename type="patronym" xml:id="{0}">{1}</forename>'
        for name_info in names[::-1]:
            person_part = par_tag.string[name_info['start']:name_info['end']]
            patterns_repls = []
            pattern_repl = pattern_repl_name_fio('first', name_info['fio'], name_format)
            if pattern_repl is not None:
                patterns_repls.append((*pattern_repl, False))

            pattern_repl = pattern_repl_name_fio('last', name_info['fio'], surname_format)
            if pattern_repl is not None:
                patterns_repls.append((*pattern_repl, False))

            pattern_repl = pattern_repl_name_fio('middle', name_info['fio'], middlename_format)
            if pattern_repl is not None:
                patterns_repls.append((*pattern_repl, False))

            name_parts = re.split('\s+', person_part)
            for i in range(len(name_parts)):
                for j in range(len(patterns_repls)):
                    pattern, repl, used = patterns_repls[j]
                    if used:
                        continue
                    if re.search(pattern, name_parts[i]):
                        name_parts[i] = re.sub(pattern, repl, name_parts[i])
                        patterns_repls[j] = (pattern, repl, True)
                        break

            new_person_part = person_format.format(location_text, ' '.join(name_parts))

            par_tag.string = par_tag.string[:name_info['start']] +\
                            new_person_part +\
                            par_tag.string[name_info['end']:]
    
    @staticmethod
    def set_dates(par_tag):
        """Расставляет теги дат"""

        def set_date_part(res_lst, date_part_type):
            if date_part_type:
                res_lst.append(str(date_part_type))
            else:
                res_lst.append('')                

        dates = extract_dates(str(par_tag.string))
        date_format = '<date when="{0}">{1}</date>'
        for date_info in dates[::-1]:
            date_part = par_tag.string[date_info.start:date_info.stop]
            res_lst = []
            set_date_part(res_lst, date_info.fact.year)
            set_date_part(res_lst, date_info.fact.month)
            set_date_part(res_lst, date_info.fact.day)
            new_date_part = date_format.format('-'.join(res_lst), date_part)
            par_tag.string = par_tag.string[:date_info.start] +\
                            new_date_part +\
                            par_tag.string[date_info.stop:]

    @staticmethod
    def process_all_existences(par_tag, location_text):
        """Полная обработка сущностей"""
        ExistenceSeter.set_people(par_tag, location_text)
        ExistenceSeter.set_dates(par_tag)

In [ ]:
class TextSetter():
    """Класс с методами для заполнения текста произведения"""

    def __init__(self, soup_html, soup_tei, soup_notes_html):
        self.soup_html = soup_html
        self.soup_tei = soup_tei
        self.soup_notes_html = soup_notes_html


    def _help_get_id(self):
        """Получает id нашего текста"""
        self._text_id = self.soup_html.find_all('a')[1].get('href').split('#')[1]


    def _help_find_text_tags(self):
        """Создает переменные-теги, отвечающие за произведение"""
        self._pars = self.soup_html.find_all(['p', 'span', 'img'])
        self._body = self.soup_tei.select('text body')[0]
        first_page = self.soup_html.find('span', class_='page').get('id')[2:]
        self._pb_tag = self.soup_tei.new_tag('pb', attrs={'n':first_page})


    def process_page(self, cur_tag):
        """Обрабатывает абзац-страницу"""
        cur_page = cur_tag.get('id')[2:]
        self._pb_tag = self.soup_tei.new_tag('pb', attrs={'n':cur_page})
        self._pb_tag.string = cur_tag.get('id')[2:]
        self._body.append(self._pb_tag)


    def move_text_with_notes(self, tag_to, tag_from):
        """Переносит содержиомое тега с сохранением сносок"""
        def help_recursive(tag_from):
            if isinstance(tag_from, str):
                return tag_from
            res = ''
            note_format = '<note xml:id={0}>{1}</note>'
            for tag_part in tag_from.contents:
                if isinstance(tag_part, str):
                    res += tag_part
                elif tag_part.name == 'a':
                    if tag_part.has_attr('class') and tag_part['class'][0] == 'footnote':
                        res += note_format.format(tag_part['href'][1:], tag_part.text)
                    elif tag_part.has_attr('href') and re.match(r'#$\S*', tag_part['href'][0]) is None:
                        res += note_format.format('external_link', tag_part.text)
                elif tag_part.name not in ['p', 'div']:
                    res += help_recursive(tag_part)
            return res

        tag_to.string = help_recursive(tag_from)
        

    def process_image(self, cur_tag):
        """Обрабатывает абзац-изображение"""
        fig_tag = self.soup_tei.new_tag('figure')
        self._pb_tag.append(fig_tag)
        graph_tag = self.soup_tei.new_tag('graphic')
        fig_tag.append(graph_tag)
        get_href = cur_tag.get('src').split('/')
        fig_tag.find('graphic')['url'] = f'http://feb-web.ru/feb/chekhov/{get_href[-2]}/{get_href[-1]}'
        desc_tag = self.soup_tei.new_tag('figDesc')
        fig_tag.append(desc_tag)
        #desc_tag.string = cur_tag.get('alt')
        self.move_text_with_notes(desc_tag, cur_tag.get('alt'))
        ExistenceSeter.process_all_existences(desc_tag, 'figure')

    
    def process_subheading(self, cur_tag):
        """Обрабатывает абзац-подзаголовок"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p)
        tag_headline = self.soup_tei.new_tag('head')
        new_p.append(tag_headline)
        self._body.find('head')['rend'] = 'center'
        self._body.find('head')['type'] = 'subtitle'
        #tag_headline.string = cur_tag.text.split('\n')[0]
        self.move_text_with_notes(tag_headline, cur_tag)
        ExistenceSeter.process_all_existences(tag_headline, 'subtitle')


    def process_heading(self, cur_tag):
        """Обрабатывает абзац-заголовок"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p)
        tag_headline = self.soup_tei.new_tag('head')
        new_p.append(tag_headline)
        self._body.find('head')['rend'] = 'center'
        tag_hi = self.soup_tei.new_tag('hi') 
        tag_headline.append(tag_hi)
        self._body.find('hi')['rend'] = 'strong'
        #tag_hi.string = cur_tag.text.split('\n')[0]
        self.move_text_with_notes(tag_hi, cur_tag)
        ExistenceSeter.process_all_existences(tag_hi, 'title')


    def process_epigraph(self, cur_tag):
        """Обрабатывает абзац-эпиграф"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p)
        tag_epigraph = self.soup_tei.new_tag('epigraph')
        new_p.append(tag_epigraph)
        self._body.find('epigraph')['rend'] = 'right'
        #tag_epigraph.string = cur_tag.text
        self.move_text_with_notes(tag_epigraph, cur_tag)
        ExistenceSeter.process_all_existences(tag_epigraph, 'epigraph')


    def process_greeting(self, cur_tag):
        """Обрабатывает абзац-приветствие/обращение"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p)
        tag_salute = self.soup_tei.new_tag('salute')
        new_p.append(tag_salute)
        #tag_salute.string = cur_tag.text.split('\n')[0]
        self.move_text_with_notes(tag_salute, cur_tag)
        ExistenceSeter.process_all_existences(tag_salute, 'salute')


    def process_signature(self, cur_tag):
        """Обрабатывает абзац-подпись"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p)
        tag_podp = self.soup_tei.new_tag('signed')
        new_p.append(tag_podp)
        #tag_podp.string = cur_tag.text.split('\n')[0]
        self.move_text_with_notes(tag_podp, cur_tag)
        ExistenceSeter.process_all_existences(tag_podp, 'signed')


    def process_text(self, cur_tag):
        """Обрабатывает абзац-текст"""
        new_p = self.soup_tei.new_tag('p')
        self._pb_tag.append(new_p) 
        #new_p.string = cur_tag.text.split('\n')[0]
        self.move_text_with_notes(new_p, cur_tag)
        ExistenceSeter.process_all_existences(new_p, 'text')


    def process_notes(self):
        """Обработка примечаний"""
        self._link_tag = self.soup_tei.new_tag('linkGrp')
        self._body.append(self._link_tag)
        self._link_tag.string = 'Примечания'
        try:
            first_tag = self.soup_notes_html.find('h4', attrs={'id':self._text_id})
            next_tag = first_tag.find_next('h4')

            note_tag = self.soup_tei.new_tag('link')
            note_tag['target'] = 'external_snos'
            self._link_tag.append(note_tag)
        except:
            return
        all_tags = first_tag.find_all_next()
        for cur_tag in all_tags:
            if cur_tag == next_tag:
                break
            if cur_tag.has_attr('class') and re.match(r'small\S*|prim\S*|text\S*|txt\S*|mtext\S*', cur_tag['class'][0]) is not None:
                if cur_tag.text[:4].lstrip() == 'Стр.' or cur_tag.text[:3].lstrip() == '...':
                    note_tag1 = self.soup_tei.new_tag('link')
                    note_tag1['target'] = 'external_snos'
                    self._link_tag.append(note_tag1)
                    new_p = self.soup_tei.new_tag('p')
                    note_tag1.append(new_p)
                    new_p.string = cur_tag.text.split('\n')[0]
                    ExistenceSeter.process_all_existences(new_p, 'note')
                else:
                    new_p = self.soup_tei.new_tag('p')
                    note_tag.append(new_p)
                    new_p.string = cur_tag.text.split('\n')[0]
                    ExistenceSeter.process_all_existences(new_p, 'note')                    
                
            
    def process_snos(self):
        """Получает примечания внутри произведения из раздела сноски"""
        text = self.soup_html.find_all('p', class_=['snos', 'snoska'])
        for a in text:
            note_tag = self.soup_tei.new_tag('link')
            note_tag['target'] = a['id']
            self._link_tag.append(note_tag)
            note_tag.string = re.sub('^\d+ ', '', a.text)
            ExistenceSeter.process_all_existences(note_tag, 'note')


    def link_notes_in_text(self):
        """Присваиваем примечаниям id"""
        notes = self.soup_tei.find('body').find_all('note')
        linkgrp = self.soup_tei.find('linkgrp')
        links_snos = linkgrp.find_all('link', target=lambda x: x != 'external_snos')
        links_external = linkgrp.find_all('link', target='external_snos')
        for note in notes:
            if note['xml:id'] == 'external_link':
                for i_link in range(len(links_external)-1, -1,-1):
                    if links_external[i_link].text.find(note.text.strip()) != -1:
                        note['xml:id'] = f'note{i_link+1}'
                        break
                    note['xml:id'] = 'note1'
            else:
                for i_link in range(len(links_snos)):
                    if note['xml:id'] == links_snos[i_link]['target']:
                        note['xml:id'] = 'note' + str(i_link+1+len(links_external))
                        break

        for i_link in range(len(links_external)):
            links_external[i_link]['target'] = f'#note{i_link+1}'

        for i_link in range(len(links_snos)):
            links_snos[i_link]['target'] = f'#note{i_link+1+len(links_external)}'


    def fill_all_text(self):
        """Полная обработка текста"""
        self._help_get_id()
        self._help_find_text_tags()

        for cur_tag in self._pars[1:]:
            if cur_tag.has_attr('class') and cur_tag['class'][0] == 'page': # Если это страница
                self.process_page(cur_tag)
            elif cur_tag.has_attr('alt'): # Если это изображение
                self.process_image(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'zg\S*', cur_tag['class'][0]) is not None: # Если это подзаголовок
                self.process_subheading(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'tit\S*|zag\S*', cur_tag['class'][0]) is not None: # Если это заголовок
                self.process_heading(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'epig\S*', cur_tag['class'][0]) is not None: # Если это эпиграф
                self.process_epigraph(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'obraw\S*', cur_tag['class'][0]) is not None: # Если это приветствие/обращение (в письме)
                self.process_greeting(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'podp\S*', cur_tag['class'][0]) is not None: # Если это подпись (в письме)
                self.process_signature(cur_tag)
            elif cur_tag.has_attr('class') and re.match(r'text\S*|curs\S*|small\S*|vis\S*|center\S*', cur_tag['class'][0]) is not None: # Если это текст
                self.process_text(cur_tag)

        # Переносим сноски
        self.process_notes()

        # Получаем примечания внутри произведения из раздела сноски
        self.process_snos()

        #исправляем возможные ошибки
        tei_doc = str(self.soup_tei.prettify()[2:])
        tei_doc = tei_doc.replace('&lt;', r'<')
        tei_doc = tei_doc.replace('&gt;', r'>')
        self.soup_tei = BeautifulSoup(tei_doc, 'html.parser')

        # Присваиваем примечаниям id
        self.link_notes_in_text()

In [ ]:
dictionary = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
        'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
        'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'kh',
        'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
        'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
        'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
        'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'KH',
        'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'Y','Ь':'','Э':'E',
        'Ю':'U','Я':'YA', ' ' : '_'}

def save_tei(soup_tei, save_dir, file_num):
    title = re.sub(r'[/\:*?''<>!«»—,]', '', soup_tei.find('title').string.strip())
    for key in dictionary:
        title = title.replace(key, dictionary[key])
    title = title[:15]

    #В зависимости от номера тома поменять цифру рядом с "V"
    with open(os.path.join(save_dir, f'V{volume_num}_{file_num}_{title}.xml'), 'w', encoding='utf8') as file: 
        file.write(soup_tei.prettify())

In [ ]:
for volume_num in range(1, 4):
    #volume_num = 2
    directory_html = Texts_HTML_PATH / str(volume_num)
    directory_tei = Texts_TEI_PATH / str(volume_num)
    create_dir(directory_tei)
    note_file = NOTES_Preform_PATH / f'{str(volume_num)}.html'
    
    print(f'Том {volume_num} обрабатывается')
    file_num = 1
    for filename in os.listdir(directory_html):
        #filename = '56_Mamasha.html'
        print(filename)
        soup_html = open_with_soup(directory_html / filename, 'windows-1251')
        soup_tei = open_with_soup(TEI_Preform_file, 'utf8')
        soup_notes_html = open_with_soup(note_file, 'windows-1251')

        #заполянем мета-информацию
        meta_setter = MetaSetter(soup_html, soup_tei)
        meta_setter.fill_all_meta(your_name)

        #заполняем текст и сноски
        text_setter = TextSetter(soup_html, soup_tei, soup_notes_html)
        text_setter.fill_all_text()

        soup_tei = text_setter.soup_tei

        #сохраняем
        save_tei(soup_tei, directory_tei, file_num)
        file_num += 1
    print('------------')